# Step 1:  Implement the Histogram of Curvature Scale

Write a function called HoCS that returns a histogram of curvature scale feature vector for a given region.  The inputs to your function should be:

- `B`: a binary image that contains exactly one foreground connected component.
- `min_scale`: The smallest scale (circle radius) at which to calcluate curvature
- `max_scale`: The largest scale (circle radius) at which to calculate curvature
- `increment`: The increment at which intermediate curvatures should be calculated (must be a positive integer)
- `num_bins`: The number of bins in the histogram of curvature for a single scale (must be a positive integer)

Your function should compute a histogram of curvature for each scale, starting at `min_scale` ending at (at most) `max_scale`, and for intermediate scales at increments of `increment`.  For example, if `min_scale`=4 and `max_scale`=20, and `increment`=3, then the function should compute a histogram of curvature for scales 4, 7, 10, 13, 16, and 19.  Each histogram at each scale should have `num_bins` bins.  Curvature must be computed using the normalized area integral invariant method described on Slide 39 of the Topic 9 lecture notes.  

Normalize each histogram at each scale.

To keep things straightforward, your functions hould only consider the outer perimeter of the input region; ignore the boundaries of holes in the region.

After computing the histogram of curvature at each of the specified scales, all of the histograms should be concatenated into a single one-dimensional array (feature vector) and then returned.

_Implementation hint:  You can calculate the normalized area integral invariant of each pixel efficiently using linear filtering.  You will find the function `skimage.morphology.disk()` function useful for designing the appropriate filter masks._

_Implementation hint:  Most of the heavy lifting here can be done with module functions from `skimage`, `numpy`, and `scipy`.  Many of the functions mentioned in class and in the notes will be useful.  One that we might not have covered, but will be very handy is `numpy.histogram()`.  When you use it, makes sure you specify both the `bins` and `range` optional arguments. Also note that `numpy.histogram()` returns TWO things.  You only need the first one, so make sure you write your function call like this:_

`the_histogram, stuff_you_dont_need = np.histogram(...)`


In [2]:
def normalize(v):
#    norm1 = x / np.linalg.norm(x)
#    norm2 = normalize(x[:, np.newaxis], axis=0).ravel()
#    return np.all(norm1 == norm2)
    return (v - min(v))/(max(v)-min(v))

def HoCS(B, min_scale, max_scale, increment, num_bins):
    '''
     Computes a histogram of curvature scale for the shape in the binary image B.
    Boundary fragments due to holes are ignored.
    :param B: A binary image consisting of a single foreground connected component.
    :param min_scale: smallest scale to consider (minimum 1)
    :param max_scale: largest scale to consider (max_scale > min_scale)
    :param increment:  increment on which to compute scales between min_scale and max_scale
    :param num_bins: number of bins for the histogram at each scale
    :return: 1D array of histograms concatenated together in order of increasing scale.
    '''

    #print(B)
    #outline = np.logical_and(B,morph.binary_erosion(B))  # Get the border pixels of the binary image
    outline = seg.find_boundaries(B, connectivity=2, mode='inner')
    #outlin = outline(B)
    #print("outline:", outline)
    # Save all the outline coordinates
    o_index = np.argwhere(outline == True)  # 2d array of two columns, row# and col# in each column
    #print(B[o_index[0][0]][o_index[0][1]])
    #print(o_index)
    # Create a 2d histogram to save of the histogram of curvature scales
    scales = int(((max_scale - min_scale) / increment) + 1)
    final_array = np.array([])
    #i = 0
    pd = 500
    B2 = np.pad(B,((pd,pd),(pd,pd)),'edge')

    # Loop through each scale
    for scale in range(min_scale, max_scale+1, increment):
        # print(o_index[:][0])
        hist_cur = np.zeros(len(o_index))
        disk = morph.disk(scale, dtype=bool)
        num_pix = np.count_nonzero(disk == 1)
        for pix in range(0, len(o_index)):
            # calculate how many 1 pixels in radius at o_index[pix][0],o_index[pix][1]
            # Calculate the normalized area integral invariant
            l = o_index[pix][0] - scale  # left
            r = o_index[pix][0] + scale  + 1# right
            u = o_index[pix][1] - scale  # above
            d = o_index[pix][1] + scale  + 1# below
            #print("l:", l, "r:", r, "u:", u, "d:", d)
            #print(disk)
            nbrhd = B2[l+pd:r+pd,u+pd:d+pd]  # neighboard about the outline pixel
            #print(nbrhd)
            overlap = np.logical_and(nbrhd, disk)  # fg pixels in the circle
            fg = np.count_nonzero(overlap == 1)
            kp = fg / (num_pix)  # normalized area integral invarient
            hist_cur[pix] = kp
            #print("fg:", fg, "kp:", kp)
            #if (pix == len(o_index)-1):
            #    hist_cur = normalize(hist_cur)

        # Save the histogram of curvature for this scale
    #    print(hist_cur)
        hist, other_stuff = np.histogram(hist_cur, bins=num_bins, range=(0.0,1.0))
    #    print(hist)
        #divide the histogram by the sum of all the bins
        final_array = np.concatenate([final_array,hist])
        #np.delete(final_array,0)
        #i += 1
    # reshape to a 1d array, this will be the concatted histograms
    return final_array / int(o_index.shape[0])

# Step 2: Test your HoCS function.

Run HoCS on `threshimage_0001.png` from the ground truth for assignment 3.  Use `min_scale=5`, `max_scale=25`, `increment=10`, `num_bins=10`.  Plot the resulting feature vector as a bar graph.  Set the y-axis limits to be between 0.0 and 1.0.  You should get a result that matches the sample output in the assignment description.


In [ ]:
import skimage.io as io
import skimage.util as util
import matplotlib.pyplot as plt
#% matplotlib inline

B = util.img_as_bool(io.imread('leaftraining/threshimage_0001.png'))
y = HoCS(B, 5, 25, 10, 10)
x = range(0,len(y))
plt.bar(x,height=y)
plt.ylim(0,1.0)
#plt.show()

# Step 3: Calculate training features.

Use your function from Step 1 to compute the HoCS feature for each of the training images.  Use them to train a k-nearest neigbour classifier.  It is up to you to determine the parameters for the HoCS feature such as `min_scale`, `max_scale`, etc. to maximize the classification rate.  This will require some experimentation.  Slides 19-21 of Topic 12 lecture notes will be helpful here.

Also generate the training labels here (a column-array of numbers indicating which descriptors belong to each class, e.g. use values 1,2,3 to indicate class 1, 2, and 3.).

In [181]:
import sklearn.neighbors as neigh
import os as os

import glob
import os
import random

images = [os.path.basename(x) for x in glob.glob('leaftraining/*.png')]
images.sort()
length = len(images)

min = 5
max = 25
inc = 8
bins = 5

hocs_feat = []
for i in range(0, length):
    B = util.img_as_bool(io.imread('leaftraining/' + images[i]))
    hocs = HoCS(B, min, max, inc, bins)
    hocs_feat.append(hocs)
hocs_feat = np.asarray(hocs_feat)

labels = np.zeros(length)
clazz = 1
for i in range(0,length):
    if (i==10):
        clazz = 2
    elif (i==20):
        clazz = 3
    labels[i] = clazz

# Step 4: Train the KNN classifier using the feature vectors from the training images.

You have another opportunity here to optimize parameters.  You can experiment with the options for the KNN classifier (in partiuclar n_neighbors) to try to obtain better classification rates.  But you won't really be able to do this until after step 6, so just use default parameters to start with. 

Hint: The steps in this notebook are broken up the way they are so that you can adjust the parameters of training the classifier and then go and perform the classfication without having to re-run the calculation of the features in steps 3 and 5.  You can adjust the parameters here in step 4, and then go and re-run the test set in Step 6 without running step 5 over again -- which is good because step 5 will take a while to run.  Of course you will have to recalculate the features each time you restart PyCharm or the Jupyter Notebook server.

In [182]:
# Train the KNN classifier

import sklearn.neighbors as neighbors
neighbs = 3
knn = neighbors.KNeighborsClassifier(n_neighbors=neighbs)
knn.fit(hocs_feat, labels)

# Step 5: Calculate the testing features.

Compute the HoCS features for all of the testing images.  Use the same HoCS parameters you did in Step 3.  Also generate class labels for the testing image descriptors.

In [183]:
# again use os.walk() to process the testing images
images = [os.path.basename(x) for x in glob.glob('leaftesting/*.png')]
length = len(images)

hocs_feat = []
for i in range(0, length):
    B = util.img_as_bool(io.imread('leaftesting/' + images[i]))
    hocs = HoCS(B, min, max, inc, bins)
    hocs_feat.append(hocs)
hocs_feat = np.asarray(hocs_feat)

labels = knn.predict(hocs_feat)

# Step 6: Classfiy the testing features.

Classify the testing image features.

Determine the classification rate and the confusion matrix by comparing the results of the classifier to the true class labels for each image.  

Print out the filenames of incorrectly classified images.

Print the confusion matrix (you don't have to print the row/column indicies as in the example in the assignment description), just the rows and columns of the matrix itself.

Print the correct classification rate.

It should be very easy to get a classficiation rate more than 90%; with care you should be able to get as much as 95%.

In [1]:
# Write your code for Step 6 here.

true_labels = np.zeros(length)
clazz = 1
for i in range(0,length):
    if (i==50):
        clazz = 2
    elif (i==77):
        clazz = 3
    true_labels[i] = clazz

confusion = np.zeros((clazz,clazz))
for i in range(0, len(true_labels)):
    print("lt", true_labels[i], "l", labels[i])
    lt = int(true_labels[i])
    l = int(labels[i])
    confusion[lt-1, l-1] += 1

print("Confusion Matrix\n", confusion)

print("Classification rate: ", np.trace(confusion)/(np.sum(confusion)/2) * 100)


NameError: name 'np' is not defined